In [ ]:
import torch
from models.vit import ViT
from models.ViT_CNN import ViT_CNN
from models.ViT_CNN_early_exit import ViT_CNN_early_exit
from functions import train_test
from functions import plotter
from functions import helpers
from functions import data_loader

In [ ]:
torch.manual_seed(42)

CIFAR-100


In [ ]:
train_loader, test_loader = data_loader.get_data_loader(
    80, 2, "datasets/cifar-100/cifar-100-python", download=True
)

CNN+ViT

In [ ]:
base_model = ViT(
    image_size=32,
    patch_size=4,
    num_classes=100,
    dim=64,
    depth=4,
    heads=8,
    mlp_dim=256,
    dropout=0.1,
).to(
    "cpu"
)  # cpu for feature visualisation
base_model_load_path = f"save_model/cifar-100/vit_base/best_model.pt"
print(f"Total parameters:{helpers.count_parameters(base_model)}")
data_loader.load_checkpoint(base_model, base_model_load_path)
base_model_acc = train_test.test(base_model, test_loader, "cuda")
base_model_loss_list, base_model_accuracy_list = data_loader.load_lists_from_file(
    "save_model/cifar-100/vit_base/loss_and_accuracy"
)
plotter.plot_loss_accuracy(base_model_loss_list, base_model_accuracy_list, "base_model")

CNN pre ViT feautre extraction + CNN patch embedding

In [ ]:
ViT_CNN_model = ViT_CNN(
    image_size=32,
    patch_size=4,
    num_classes=100,
    dim=64,
    depth=2,
    heads=4,
    mlp_dim=256,
    dropout=0.1,
).to(
    "cpu"
)  # cpu for feature visualisation
ViT_CNN_model_load_path = "save_model/cifar-100/ViT_CNN/best_model.pt"
print(f"Total parameters:{helpers.count_parameters(ViT_CNN_model)}")
data_loader.load_checkpoint(ViT_CNN_model, ViT_CNN_model_load_path)
ViT_CNN_model_acc = train_test.test(ViT_CNN_model, test_loader, "cuda")
ViT_CNN_model_loss_list, ViT_CNN_model_accuracy_list = data_loader.load_lists_from_file(
    "save_model/cifar-100/ViT_CNN/loss_and_accuracy"
)
plotter.plot_loss_accuracy(
    ViT_CNN_model_loss_list, ViT_CNN_model_accuracy_list, "ViT_CNN_model"
)

In [ ]:
plotter.plot_accuracy_comparison(
    [ViT_CNN_model_accuracy_list, base_model_accuracy_list],
    ["ViT_CNN_model", "ViT_base_model"],
)

In [ ]:
data_loader.create_comparison_table(
    [base_model, ViT_CNN_model],
    [base_model_acc, ViT_CNN_model_acc],
    ["ViT_base_model", "ViT_CNN_model"],
)

In [ ]:
base_model.to("cpu")
x, img = data_loader.get_random_image("bee")
plotter.plot_feature_maps(base_model, x, img, device="cpu")

In [ ]:
ViT_CNN_model.to("cpu")
plotter.plot_feature_maps(ViT_CNN_model, x, img, device="cpu")

Knowledge distillation

Student base model

In [ ]:
student_base_model = ViT(
    image_size=32,
    patch_size=4,
    num_classes=100,
    dim=64,
    depth=3,
    heads=6,
    mlp_dim=256,
    dropout=0.1,
).to("cuda")
student_base_load_path = "save_model/cifar-100/vit_student_base/best_model.pt"
print(f"Total parameters:{helpers.count_parameters(student_base_model)}")
data_loader.load_checkpoint(student_base_model, student_base_load_path)
student_base_model_acc = train_test.test(student_base_model, test_loader, "cuda")
(
    student_base_model_loss_list,
    student_base_model_accuracy_list,
) = data_loader.load_lists_from_file(
    "save_model/cifar-100/vit_student_base/loss_and_accuracy"
)
plotter.plot_loss_accuracy(
    student_base_model_loss_list, student_base_model_accuracy_list, "student_base_model"
)

Response based kd


In [ ]:
student_response_kd = ViT(
    image_size=32,
    patch_size=4,
    num_classes=100,
    dim=64,
    depth=3,
    heads=6,
    mlp_dim=256,
    dropout=0.1,
    feature_distill=False,
).to("cuda")
student_response_kd_path = "save_model/cifar-100/vit_student_kd/best_model.pt"
print(f"Total parameters:{helpers.count_parameters(student_response_kd)}")
data_loader.load_checkpoint(student_response_kd, student_response_kd_path)
student_kd_acc = train_test.test(student_response_kd, test_loader, "cuda")
student_kd_loss_list, student_kd_accuracy_list = data_loader.load_lists_from_file(
    "save_model/cifar-100/vit_student_kd/loss_and_accuracy"
)
plotter.plot_loss_accuracy(student_kd_loss_list, student_kd_accuracy_list, "student_kd")

Feature based kd

Combined response+feature kd

In [ ]:
plotter.plot_accuracy_comparison(
    [student_kd_accuracy_list, student_base_model_accuracy_list,base_model_accuracy_list],
    ["student_kd", "student_base_model","base_model"],
)

In [ ]:
data_loader.create_comparison_table(
    [
        base_model,
        student_base_model,
        student_response_kd,
    ],
    [base_model_acc,student_base_model_acc,student_kd_acc,],
    ["base_model","student_kd", "student_base_model"],
)

In [ ]:
# Add inference time comparison and co2 eq

ViT_CNN early exit


In [ ]:
ViT_CNN_early_exit_ = ViT_CNN_early_exit(
    image_size=32,
    dim=64,
    patch_size=4,
    num_classes=100,
    depth=4,
    heads=8,
    mlp_dim=256,
    dropout=0.1,
    early_exit=False,
).to("cuda")
ViT_early_exit_load_path = "save_model/cifar-100/ViT_CNN_early_exit3/best_model.pt"
print(f"Total parameters:{helpers.count_parameters(ViT_CNN_early_exit_)}")
data_loader.load_checkpoint(ViT_CNN_early_exit_, ViT_early_exit_load_path)

In [ ]:
ViT_early_exitacc = train_test.test(ViT_CNN_early_exit_, test_loader, "cuda")


In [ ]:
ViT_CNN_early_exit = ViT_CNN_early_exit(
    image_size=32,
    dim=64,
    patch_size=4,
    num_classes=100,
    depth=4,
    heads=8,
    mlp_dim=256,
    dropout=0.1,
    early_exit=True,
).to("cuda")
ViT_early_exit_load_path = "save_model/cifar-100/ViT_CNN_early_exit3/best_model.pt"
print(f"Total parameters:{helpers.count_parameters(ViT_CNN_early_exit)}")
data_loader.load_checkpoint(ViT_CNN_early_exit, ViT_early_exit_load_path)

In [ ]:
train_loader_1_example, test_loader_1_example = data_loader.get_data_loader(
    1, 2, "datasets/cifar-100/cifar-100-python", download=True
)


In [ ]:

ViT_CNN_early_exit_acc, num_early_exits = train_test.test(
    ViT_CNN_early_exit, test_loader_1_example, "cuda"
)



In [ ]:
# plotter.plot_accuracy_comparison(
#     [base_model_accuracy_list, ViT_CNN_early_exit_accuracy_list],
#     ["base_model", "ViT_CNN_early_exit"],
# )

In [ ]:
data_loader.create_comparison_table(
    [base_model, ViT_CNN_early_exit], [base_model_acc, ViT_CNN_early_exit_acc],
    ['base_model','ViT_CNN_early_exit']
)